# Workout Embeddings

This notebook uses the OpenAI API to generate embeddings for workout descriptions. Then it makes some nice interactive plots.

In [1]:
from os import walk
from os.path import join, expanduser
import openai
import numpy as np
import pandas as pd
import json
import textwrap
from scipy.spatial import distance_matrix
from sklearn import manifold, cluster

In [2]:
from bokeh.plotting import curdoc, figure, show
from bokeh import palettes
from bokeh.transform import linear_cmap
from bokeh.models import ColumnDataSource, Span, Label
from bokeh.embed import components
from bokeh.io import output_notebook, save, output_file

## Config

In [3]:
# whether to save plots or show them in the notebook
saving = True
# output directory for embeddings
outdir = join("..", "data", "processed", "embeddings")
# output directory for plots/images
plotdir = join("..", "plots", "embeddings")

In [4]:
# plotting config
# curdoc().theme = "dark_minimal"
if not saving:
    output_notebook()

In [5]:
def save_or_show(f, fn):
    if saving:
        # stand alone document
        output_file(join(plotdir, f"standalone_{fn}.html"))
        save(f)
        # components
        script, div = components(f)
        with open(join(plotdir, f"{fn}.js"), "w") as ofile:
            ofile.write(script)
        with open(join(plotdir, f"{fn}.html"), "w") as ofile:
            ofile.write(div)
    else:
        show(f)

In [6]:
with open(join(expanduser("~"), "openai_api_key"), "r") as ifile:
    api_key = api_key = ifile.read().strip()
    client = openai.OpenAI(api_key=api_key)

## load data

In [7]:
# read workout descriptions
workouts = []
for root, _, fns in walk(join("..", "data", "raw", "workouts")):
    for fn in filter(lambda fn: ".txt" in fn, fns):
        with open(join(root, fn), "r") as ifile:
            desc = ifile.read()
        s = fn.replace(".txt", "").split("_")
        workouts.append([s[0], int(s[1]), int(s[-1]), desc])
workouts = pd.DataFrame(
    workouts, columns=["competitionType", "year", "workoutNumber", "workoutDescription"]
)
workouts = workouts.set_index(["year", "competitionType", "workoutNumber"]).sort_index()
workouts

workoutDescription
year competitionType workoutNumber                                                   
2007 games           1              For time:\n1,000-meter row\nThen, 5 rounds of:...
                     2                                Trail run (approximately 5 k)\n
                     3              Back squat 1-rep max\nPress 1-rep max\nDeadlif...
2008 games           1              21-15-9 reps for time of:\nThrusters (95/65 lb...
                     2              5 rounds for time of:\n5 Deadlifts (275/185 lb...
...                                                                               ...
2023 open            3                          1-rep-max thruster (from the floor)\n
                     4              Starting with a 6-minute time cap, complete as...
2024 open            1              21 dumbbell snatches, arm 1\n21 lateral burpee...
                     2              As many rounds and reps as possible in 20 minu...
                     3              All for time:\n\n5 rounds of:\n    10 thruster...

[268 rows x 1 columns]

In [8]:
# read the control table, for workout names
controls = pd.read_csv("../data/processed/controls.csv")
controls = controls[controls.divisionNumber == 1].drop(
    columns=["divisionName", "divisionNumber"]
)
controls = controls.set_index(["year", "competitionType", "workoutNumber"]).sort_index()
controls

workoutName
year competitionType workoutNumber                   
2007 games           1                         Hopper
                     2                      Trail Run
                     3                 CrossFit Total
2008 games           1                           Fran
                     2              Deadlift / Burpee
...                                               ...
2021 games           15                      Event 15
     open            1                           21.1
                     2                           21.2
                     3                           21.3
                     4                           21.4

[233 rows x 1 columns]

In [9]:
data = pd.merge(workouts, controls, how="left", left_index=True, right_index=True)
data = data.reset_index()
data = data.sort_values(["year", "competitionType", "workoutNumber"])
# fill empty workout names
for i in range(len(data)):
    if pd.isnull(data.at[i, "workoutName"]):
        row = data.loc[i]
        data.at[i, "workoutName"] = (
            f"{row['competitionType']} {row['year']}, Event {row['workoutNumber']}".title()
        )
data

,year,competitionType,workoutNumber,workoutDescription,workoutName
0,2007,games,1,"For time:\n1,000-meter row\nThen, 5 rounds of:...",Hopper
1,2007,games,2,Trail run (approximately 5 k)\n,Trail Run
2,2007,games,3,Back squat 1-rep max\nPress 1-rep max\nDeadlif...,CrossFit Total
3,2008,games,1,21-15-9 reps for time of:\nThrusters (95/65 lb...,Fran
4,2008,games,2,5 rounds for time of:\n5 Deadlifts (275/185 lb...,Deadlift / Burpee
...,...,...,...,...,...
263,2023,open,3,1-rep-max thruster (from the floor)\n,"Open 2023, Event 3"
264,2023,open,4,"Starting with a 6-minute time cap, complete as...","Open 2023, Event 4"
265,2024,open,1,"21 dumbbell snatches, arm 1\n21 lateral burpee...","Open 2024, Event 1"
266,2024,open,2,As many rounds and reps as possible in 20 minu...,"Open 2024, Event 2"


## formatting

In [10]:
def format_desc(desc):
    return "\n".join([textwrap.fill(line, width=60) for line in desc.split("\n")])

In [11]:
data["workoutDescription"] = data["workoutDescription"].apply(format_desc)
data.head(3)

,year,competitionType,workoutNumber,workoutDescription,workoutName
0,2007,games,1,"For time:\n1,000-meter row\nThen, 5 rounds of:...",Hopper
1,2007,games,2,Trail run (approximately 5 k)\n,Trail Run
2,2007,games,3,Back squat 1-rep max\nPress 1-rep max\nDeadlif...,CrossFit Total


## embed workout descriptions

In [12]:
def get_embedding(text, model="text-embedding-3-large"):
    response = client.embeddings.create(input=text, model=model)
    return np.array(response.data[0].embedding)

In [13]:
data["embedding"] = data.workoutDescription.apply(get_embedding)

In [14]:
data = data.sort_values(["competitionType", "year", "workoutNumber"])
data.sample(10)

,year,competitionType,workoutNumber,workoutDescription,workoutName,embedding
48,2012,games,9,"Five rounds for time of:\n20 foot Rope climb, ...",Rope-Sled,"[-0.03380583971738815, -0.02947869338095188, -..."
166,2018,open,1,Complete as many rounds as possible in 20 minu...,18.1,"[-0.037518471479415894, -0.05463021993637085, ..."
229,2021,open,2,10 dumbbell snatches\n15 burpee box jump-overs...,21.2,"[0.02034907229244709, -0.03267975524067879, -0..."
216,2021,games,4,For time:\n\n10-9-8-7-6-5-4-3-2-1 reps of:\n\n...,Event 4,"[-0.0389130674302578, -0.020575715228915215, -..."
193,2020,games,5,5 rounds:\n500-m run\n15 overhead squats\n15 b...,Nasty Nancy,"[-0.044030215591192245, -0.047732628881931305,..."
71,2013,games,12,Three rounds of:\n5 Muscle-ups\n5 Deficit hand...,The Cinco 2,"[-0.019886117428541183, -0.018392043188214302,..."
104,2015,games,10,15-10-6 reps for time of:\n\nThrusters (165 / ...,Triangle Couplet,"[-0.0006967788212932646, -0.042338646948337555..."
131,2016,open,3,Complete as many rounds and reps as possible i...,16.3,"[-0.007047910708934069, -0.016719451174139977,..."
127,2016,games,14,For time:\n200-m SkiErg\n50/40 double-unders\n...,Rope Chipper,"[0.0014809586573392153, -0.03054065816104412, ..."
144,2017,games,11,5 rounds for time of:\nRun 450 meters\n7 hay b...,Madison Triplet,"[-0.014892223291099072, -0.058138322085142136,..."


## embedding distances

In [15]:
# stack all the embeddings together, in order, for workouts and entire events
X = np.stack(data["embedding"])
print(X.shape)
# then compute a distance matrix between them all
D = distance_matrix(X, X)
print(D.shape)

(268, 3072)
(268, 268)


## similarity matrix

In [16]:
def frame_heatmap(D, df):
    df = df.copy().rename(
        columns=dict(
            competitionType="competition",
            workoutNumber="number",
            workoutName="name",
            workoutDescription="description",
        )
    )
    df = df.copy()
    D = D.copy()
    for i in range(D.shape[0]):
        D[i, i] = np.nan

    label_cols = [
        "competition",
        "year",
        "number",
        "name",
        "description",
    ]

    hm = pd.DataFrame(D).reset_index()
    for col in label_cols:
        hm[col] = df[col].values
    hm = hm.melt(id_vars=["index"] + label_cols, var_name="column")
    for col in label_cols:
        hm[col + "_b"] = [df[col].iat[i] for i in hm.column]
    for col in ("competition", "competition_b"):
        hm[col] = hm[col].str.title()
    return hm


def heatmap(title, width, height, tooltips, hm):
    p = figure(title=title, width=width, height=height, tooltips=tooltips)
    p.rect(
        x="column",
        y="index",
        width=1,
        height=1,
        line_alpha=0,
        fill_color=linear_cmap(
            "value",
            palette=palettes.Inferno256[::-1],
            low=hm.value.min(),
            high=hm.value.max(),
        ),
        source=ColumnDataSource(hm),
    )
    p.y_range.flipped = True
    p.xgrid.visible = False
    p.ygrid.visible = False
    return p

In [17]:
p = heatmap(
    "Embedding Distances, CrossFit Games & Open Workout Descriptions",
    1800,
    1700,
    """
    <div>
        <table>
            <tr>
                <td style="text-align: center;">Row @index</td>
                <td style="text-align: center;">Column @column</td>
            </tr>
            <tr>
                <td style="vertical-align: top; padding-right: 0.2cm; border-right: 1px solid black;">
                    <b style="font-size: 14">@competition @year</b>
                    <br>
                    <span style="font-size: 12">Event <b>@number</b>: <i>@name</i><span>
                    <hr>
                    <pre style="font-size: 10">@description</pre>
                </td>
                <td style="vertical-align: top; padding-left: 0.2cm; ">
                    <b style="font-size: 14">@competition_b @year_b</b>
                    <br>
                    <span style="font-size: 12">Event <b>@number_b</b>: <i>@name_b</i><span>
                    <hr>
                    <pre style="font-size: 10">@description_b</pre>
                </td>
            </tr>
        </table>
    </div>
    """,
    frame_heatmap(D, data),
)
N = data.shape[0]
n = data["competitionType"].value_counts()["games"]
# lines
p.add_layout(Span(location=n - 0.5, dimension="width", line_color="gray"))
p.add_layout(Span(location=n - 0.5, dimension="height", line_color="gray"))
# label settings
label_kws = dict(x_units="data", y_units="data", text_color="black")
# bottom
p.add_layout(
    Label(
        x=n / 2,
        y=N,
        text="Games",
        text_align="center",
        text_baseline="top",
        **label_kws
    )
)
p.add_layout(
    Label(
        x=n + (N - n) / 2,
        y=N,
        text="Open",
        text_align="center",
        text_baseline="top",
        **label_kws
    )
)
# top
p.add_layout(
    Label(
        x=n / 2,
        y=0,
        text="Games",
        text_align="center",
        text_baseline="bottom",
        **label_kws
    )
)
p.add_layout(
    Label(
        x=n + (N - n) / 2,
        y=0,
        text="Open",
        text_align="center",
        text_baseline="bottom",
        **label_kws
    )
)
# left
p.add_layout(
    Label(
        x=-0.5,
        y=n / 2,
        text="Games",
        text_align="right",
        text_baseline="middle",
        **label_kws
    )
)
p.add_layout(
    Label(
        x=-0.5,
        y=n + (N - n) / 2,
        text="Open",
        text_align="right",
        text_baseline="middle",
        **label_kws
    )
)
# right
p.add_layout(
    Label(
        x=N,
        y=n / 2,
        text="Games",
        text_align="left",
        text_baseline="middle",
        **label_kws
    )
)
p.add_layout(
    Label(
        x=N,
        y=n + (N - n) / 2,
        text="Open",
        text_align="left",
        text_baseline="middle",
        **label_kws
    )
)

save_or_show(p, "heatmap_combined_individual")

## clustering

In [18]:
# TSNE & clustering
tsne = manifold.TSNE(
    n_components=2,
    perplexity=12,
    # random_state=1,
    init="random",
    metric="precomputed",
    method="exact",
).fit_transform(D.copy())

In [19]:
clusters = cluster.AffinityPropagation().fit(X.copy())
ncluster = len(set(clusters.labels_))
print(f"{ncluster} clusters")

43 clusters


In [20]:
data["x"] = tsne[:, 0]
data["y"] = tsne[:, 1]
data["cluster"] = [label + 1 for label in clusters.labels_]

In [21]:
p = figure(
    width=750,
    height=750,
    tooltips="""
    <div>
        <span style="text-align: center;">Cluster @cluster</span>
        <br>
        <b>@competition @year</b>
        <br>
        Event <b>@number</b>: <i>@name</i>
        <pre style="font-size: 10">@description</pre>
    <div>
    """,
    title="TSNE, Workout Description Embeddings",
)
df = data.copy().rename(
    columns=dict(
        competitionType="competition",
        workoutNumber="number",
        workoutName="name",
        workoutDescription="description",
    )
)
df.competition = df.competition.str.title()
df["fill_color"] = [
    palettes.Turbo256[(i * 256 // ncluster) - 1] for i in data["cluster"]
]
p.scatter(
    x="x",
    y="y",
    size=12,
    fill_color="fill_color",
    line_color="gray",
    source=ColumnDataSource(df),
)
save_or_show(p, "tsne_workouts")

## cluster summarization

In [22]:
# summarize clusters and create a dictionary including workout descriptions
summaries = dict()
for n, clus in data.groupby("cluster"):
    descriptions = "\n\n".join(clus.workoutDescription.values)
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            dict(
                role="user",
                content=f"As concisely as possible, please explain what movements and exercises these workout descriptions have in common using a single incomplete sentence:\n\n{descriptions}",
            )
        ],
    )
    summary = response.choices[0].message.content
    summaries[n] = dict(cluster=n, size=len(clus), summary=summary)

In [23]:
for n, clus in data.groupby("cluster"):
    summaries[n]["year"] = list(map(int, clus["year"].values))
    for col in ["competitionType", "workoutName", "workoutDescription"]:
        summaries[n][col] = list(clus[col].values)

In [24]:
with open(join(outdir, "clusters.json"), "w") as ofile:
    json.dump(list(summaries.values()), ofile, indent=2, sort_keys=True)

In [25]:
# save the whole dataframes for later, if desired
data.to_json(
    join(outdir, "embeddings.json"),
    orient="index",
    indent=1,
    double_precision=10,
)

In [26]:
df = data.groupby(["cluster", "competitionType"]).count().reset_index()
df = (
    df[["cluster", "competitionType", "year"]]
    .rename(columns=dict(year="count"))
    .sort_values("cluster")
)
df = df.pivot(columns="competitionType", values="count", index="cluster")
df["summary"] = [summaries[idx]["summary"] for idx in df.index]
df["workouts"] = [
    textwrap.fill(" | ".join(summaries[idx]["workoutName"]), width=60)
    for idx in df.index
]
df.fillna(0, inplace=True)
df.games = df.games.astype(int)
df.open = df.open.astype(int)
df["summary"] = df["summary"].apply(lambda s: textwrap.fill(s, width=60))
df.head()

competitionType,games,open,summary,workouts
cluster,,,,
1,7,0,Each workout description includes a timed rowi...,Hopper | 2007 | Triple-3 | Second Cut | 2007 R...
2,3,4,"All workouts include rep-based, timed exercise...",Fran | Fran | Bicouplet 1 | 14.5 | 15.5 | 16.5...
3,3,0,All workouts require completing a set number o...,Squat Grace | Isabel | Double Grace
4,16,5,These workouts all include a combination of ca...,"Chipper | Pyramid Double ""Helen"" | Chipper | L..."
5,10,1,All workouts include exercises that involve ei...,Triplet | Cleans-Handstand Pushups | The Final...


In [27]:
p = figure(
    height=550,
    width=750,
    title="Workout Cluster Counts",
    x_axis_label="cluster index",
    y_axis_label="count",
    tools="hover",
    tooltips="""
    <div>
        <b>Workout Cluster $index Summary</b>
        <br>
        <pre style="font-size: 12">@summary</pre>
        <br>
        <b>Workout Names</b>
        <pre style="font-size: 10">@workouts</pre>
    <div>
    """,
)
p.vbar_stack(
    ["games", "open"],
    x="cluster",
    color=[palettes.Category10[10][0], palettes.Category10[10][1]],
    legend_label=["games", "open"],
    alpha=0.6,
    source=ColumnDataSource(df),
)
save_or_show(p, "competition_cluster_counts")